## Import & config

In [5]:
## utf-8 encoding
import os
import sys
from glob import glob
import requests
import random
import numpy as np
import pandas as pd
import cv2

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from tqdm.notebook import tqdm
from time import time
from torch.optim import *
import timm

import albumentations as A
from albumentations import *
from albumentations.pytorch import ToTensorV2

import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
timm.list_models()

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'beitv2_base_patch16_224',
 'beitv2_base_patch16_224_in22k',
 'beitv2_large_patch16_224',
 'beitv2_large_patch16_224_in22k',
 'botnet26t_256',
 'botnet50ts_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'coatnet_0_224',
 'coatnet_0_rw_224',
 'coatnet_1_224',
 'coatnet_1_rw_224',
 'coatnet_2_224',
 'coatnet_2_rw_224',
 'coatnet_3_224',
 'coatnet_3_rw_224',
 'coatnet_4_224',
 'coatnet_5_224',
 'coatnet_bn_0_rw_224',
 'coatnet_nano_cc_224',
 'coatnet_nano_rw_224',
 'coatnet_pico_rw_224',
 'coatnet_rmlp_0_rw_224',
 'coatnet_rmlp_1_rw

In [7]:
### Configurations

train_folder_dir = '/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/dataset/train'
eval_folder_dir = '/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/dataset/eval'

train_imgs_dir = f'{train_folder_dir}/images'
train_labels_path = f'{train_folder_dir}/train.csv'
eval_imgs_dir = f'{eval_folder_dir}/images'
eval_labels_path = f'{eval_folder_dir}/info.csv'

cfg = {
    'model_name' : 'efficientnet_b4',
    'epochs' : 25,
    'lr' : 1e-4,
    'seed':42,
    'num_classes':18
    }

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data Processing

### dataset

In [8]:
df = pd.read_csv(train_labels_path)
df

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59
...,...,...,...,...,...
2695,006954,male,Asian,19,006954_male_Asian_19
2696,006955,male,Asian,19,006955_male_Asian_19
2697,006956,male,Asian,19,006956_male_Asian_19
2698,006957,male,Asian,20,006957_male_Asian_20


In [9]:
## csv → label_codes, img_paths

df = pd.read_csv(train_labels_path)

def get_label(mask_tag, gender, age):
    gender = gender.lower()
    label = 0
    if mask_tag == 'Wear' and gender == 'male' and age<30: # 0:
        label = 0
    elif mask_tag == 'Wear' and gender == 'male' and 30<=age<60: # 1
        label = 1
    elif mask_tag == 'Wear' and gender == 'male' and age>=60: # 2
        label = 2
    elif mask_tag == 'Wear' and gender == 'female' and age<30: # 3
        label = 3
    elif mask_tag == 'Wear' and gender == 'female' and 30<=age<60: # 4
        label = 4
    elif mask_tag == 'Wear' and gender == 'female' and age>=60: # 5
        label = 5
    elif mask_tag == 'Incorrect' and gender == 'male' and age<30: # 6
        label = 6
    elif mask_tag == 'Incorrect' and gender == 'male' and 30<=age<60: # 7
        label = 7
    elif mask_tag == 'Incorrect' and gender == 'male' and age>=60: # 8
        label = 8
    elif mask_tag == 'Incorrect' and gender == 'female' and age<30: # 9
        label = 9
    elif mask_tag == 'Incorrect' and gender == 'female' and 30<=age<60: # 10
        label = 10
    elif mask_tag == 'Incorrect' and gender == 'female' and age>=60: # 11
        label = 11
    elif mask_tag == 'Not Wear' and gender == 'male' and age<30: # 12
        label = 12
    elif mask_tag == 'Not Wear' and gender == 'male' and 30<=age<60: # 13
        label = 13
    elif mask_tag == 'Not Wear' and gender == 'male' and age>=60: # 14
        label = 14
    elif mask_tag == 'Not Wear' and gender == 'female' and age<30: # 15
        label = 15
    elif mask_tag == 'Not Wear' and gender == 'female' and 30<=age<60: # 16
        label = 16
    elif mask_tag == 'Not Wear' and gender == 'female' and age>=60: # 17
        label = 17
    else:
        raise ValueError
    return label

label_codes = []
img_paths = []

for gender, age, folder_name in zip(df['gender'], df['age'], df['path']):
    folder_path = os.path.join(train_imgs_dir, folder_name)
    files = os.listdir(folder_path)
    for file_name in files:
        img_path = os.path.join(folder_path,file_name)
        if 'incorrect' in file_name:
            label = get_label('Incorrect', gender, age)
        elif 'mask' in file_name:
            label = get_label('Wear', gender, age)
        elif 'normal' in file_name:
            label = get_label('Not Wear', gender, age)
        else:
            raise ValueError
        img_paths.append(img_path)
        label_codes.append(label)

In [10]:
for path, code in zip(img_paths, label_codes):
    print(path, code)
    break

/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/dataset/train/images/000001_female_Asian_45/mask1.jpg 4


In [11]:
## class distribution
counts = pd.DataFrame(label_codes).value_counts().sort_index()
counts

0     2745
1     2050
2      415
3     3660
4     4085
5      545
6      549
7      410
8       83
9      732
10     817
11     109
12     549
13     410
14      83
15     732
16     817
17     109
dtype: int64

In [12]:
## data split
train_paths, valid_paths, train_labels, valid_labels = train_test_split(img_paths, label_codes,
                                                                        train_size = 0.7,
                                                                        shuffle = True,
                                                                        random_state = cfg['seed'],
                                                                        stratify=label_codes)

In [13]:
train_csv = pd.DataFrame(list(zip(train_paths, train_labels)), columns=['img_path', 'class'])

In [14]:
valid_csv = pd.DataFrame(list(zip(valid_paths, valid_labels)), columns=['img_path', 'class'])

In [17]:
print(len(train_csv), len(valid_csv))

13230 5670


In [18]:
train_csv

,img_path,class
0,/opt/ml/repo/level1_imageclassification_cv-lev...,0
1,/opt/ml/repo/level1_imageclassification_cv-lev...,16
2,/opt/ml/repo/level1_imageclassification_cv-lev...,4
3,/opt/ml/repo/level1_imageclassification_cv-lev...,4
4,/opt/ml/repo/level1_imageclassification_cv-lev...,4
...,...,...
13225,/opt/ml/repo/level1_imageclassification_cv-lev...,4
13226,/opt/ml/repo/level1_imageclassification_cv-lev...,4
13227,/opt/ml/repo/level1_imageclassification_cv-lev...,7
13228,/opt/ml/repo/level1_imageclassification_cv-lev...,3


In [19]:
## input csv, output img, label(class)
class MaskDataSet(Dataset): # train_labels_path
    def __init__(self, path_df, transforms=None):
        self.transforms = transforms
        self.path_df = path_df
        if transforms is None:
            print('If transforms param is None automatically Resize(224), Normalize, ToTensorV2 Apply')
    
    def __getitem__(self, index):
        img_path = self.path_df['img_path'][index]
        img = np.array(Image.open(img_path))
        img_class = self.path_df['class'][index]
        if self.transforms is not None:
            img = self.transforms(image=img)["image"]
        else:
            temp_transforms = A.Compose([A.Resize(height=224, width=224), A.Normalize(), ToTensorV2()])
            img = temp_transforms(image=img)["image"]
        return img, img_class

    def __len__(self):
        return len(self.path_df)

In [20]:
train_transforms = A.Compose([A.Resize(height=224, width=224),
                              A.HorizontalFlip(p=0.5),
                              A.RandomBrightnessContrast(p=0.5),
                              A.GaussianBlur(p=0.5),
                              A.GridDistortion(p=0.5),
                              A.Rotate(limit=30, p=0.5),
                              A.Normalize(mean=(0.56019358,0.52410121,0.501457),
                                          std=(0.23318603,0.24300033,0.24567522)),
                              ToTensorV2()])

valid_transforms = A.Compose([A.Resize(height=224, width=224),
                              A.Normalize(mean=(0.56019358,0.52410121,0.501457),
                                          std=(0.23318603,0.24300033,0.24567522)),
                              ToTensorV2()])

train_dataset = MaskDataSet(train_csv, train_transforms)
valid_dataset = MaskDataSet(valid_csv, valid_transforms)

### dataloader

In [21]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=True)

## Modeling

In [22]:
def get_pretrain_model(config):
    model_name = config['model_name']
    model = timm.create_model(model_name=config['model_name'], pretrained=True, num_classes=config['num_classes'])
    return model

pretrained_model = get_pretrain_model(cfg)
pretrained_model.to(device)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b4_ra2_320-7eb33cd5.pth" to /opt/ml/.cache/torch/hub/checkpoints/efficientnet_b4_ra2_320-7eb33cd5.pth


EfficientNet(
  (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNormAct2d(
          48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
    

In [23]:
## test
batch_images, batch_labels = next(iter(train_loader))
torch.argmax(pretrained_model(batch_images.to(device)), dim=-1)

tensor([15,  9, 13,  9,  3,  1, 16, 11,  8,  9,  8,  3, 13, 13, 15,  2, 15, 15,
        10, 11,  2,  2,  9, 15, 13,  2,  3, 10, 12, 10,  8, 10, 11, 11,  1,  3,
         8, 10,  8,  3,  0, 11, 17, 16, 12, 15,  8,  8,  7, 17, 11,  2, 11,  2,
        11,  7, 15, 15,  3,  2,  0, 10, 17, 15], device='cuda:0')

## Training

In [25]:
# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optim = AdamW(pretrained_model.parameters(), lr = cfg['lr'])

In [27]:
def train_and_valid(cfg, model, train_loader, valid_loader):
  result = {"train_loss" : [],
            "valid_loss" : [],
            "valid_acc":[],
            "valid_f1":[]}
  for epoch in tqdm((range(1,cfg['epochs']))):
      model.train()
      running_train_loss = []
      running_valid_loss = []

      preds = []
      labels = []


      for iter_idx, (train_imgs, train_labels) in enumerate(train_loader, start=1):
          train_imgs, train_labels = train_imgs.to(device, dtype=torch.float), train_labels.to(device)
          optim.zero_grad()
          train_pred = model(train_imgs)
          train_loss = criterion(train_pred, train_labels)
          train_loss.backward()
          optim.step()
          running_train_loss.append(train_loss.cpu().item())
  
      with torch.no_grad():
        for iter_idx, (valid_imgs, valid_labels) in enumerate(valid_loader, start=1):
          model.eval()
          valid_imgs, valid_labels = valid_imgs.to(device, dtype=torch.float), valid_labels.to(device)
          
          valid_logit = model(valid_imgs)
          valid_loss = criterion(valid_logit, valid_labels)
          valid_preds = valid_logit.argmax(dim=-1)
          
          running_valid_loss.append(valid_loss.cpu().item())
          preds += valid_preds.detach().cpu().numpy().tolist()
          labels += valid_labels.detach().cpu().numpy().tolist()

      train_loss = np.mean(running_train_loss)
      valid_loss = np.mean(running_valid_loss)
      valid_f1 = f1_score(labels, preds, average='macro')
      valid_acc = accuracy_score(labels,preds)
      print(f"{epoch}/{cfg['epochs']} : train_loss:{train_loss:.4f}, \
        valid_loss:{valid_loss:.4f}, valid_acc:{valid_acc:.2f}, valid_f1:{valid_f1:.2f}")
      result['train_loss'].append(train_loss)
      result['valid_loss'].append(valid_loss)
      result['valid_acc'].append(valid_acc)
      result['valid_f1'].append(valid_f1)
  return result, model

In [28]:
result, fine_tuned_model = train_and_valid(cfg, pretrained_model, train_loader, valid_loader)

1/25 : train_loss:1.5008,         valid_loss:0.7497, valid_acc:0.77, valid_f1:0.52
2/25 : train_loss:0.5752,         valid_loss:0.4611, valid_acc:0.84, valid_f1:0.64
3/25 : train_loss:0.3786,         valid_loss:0.3624, valid_acc:0.87, valid_f1:0.72
4/25 : train_loss:0.2854,         valid_loss:0.2926, valid_acc:0.90, valid_f1:0.78
5/25 : train_loss:0.2127,         valid_loss:0.2623, valid_acc:0.91, valid_f1:0.79
6/25 : train_loss:0.1602,         valid_loss:0.2180, valid_acc:0.93, valid_f1:0.82
7/25 : train_loss:0.1239,         valid_loss:0.1819, valid_acc:0.94, valid_f1:0.83
8/25 : train_loss:0.1014,         valid_loss:0.1766, valid_acc:0.94, valid_f1:0.84
9/25 : train_loss:0.0807,         valid_loss:0.1578, valid_acc:0.95, valid_f1:0.85
10/25 : train_loss:0.0617,         valid_loss:0.1456, valid_acc:0.95, valid_f1:0.87
11/25 : train_loss:0.0537,         valid_loss:0.1327, valid_acc:0.96, valid_f1:0.88
12/25 : train_loss:0.0491,         valid_loss:0.1421, valid_acc:0.96, valid_f1:0.88
1

## Result check

In [29]:
fine_tuned_model

EfficientNet(
  (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNormAct2d(
          48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
    

In [64]:
result

{'train_loss': [0.643201829586628,
  0.2665444292164079,
  0.18419367013778087,
  0.14639450001399873,
  0.10069840417607971,
  0.08493318321444274,
  0.0782051240026519,
  0.06409295268608751,
  0.05523639867883086,
  0.052613029242965625,
  0.052571648713831166,
  0.0475611033183545,
  0.04836737109766598,
  0.037466579966094114,
  0.041363985236095274,
  0.03674749801653001,
  0.04089784929483841,
  0.03241687317509294,
  0.03172994180242778,
  0.03126147162880082,
  0.03624531352084922,
  0.02651915400950373,
  0.031479752304230404,
  0.025083742281062086],
 'valid_loss': [0.2708845597304655,
  0.2077813320113032,
  0.15726927165569884,
  0.09815930484092972,
  0.09539370841524575,
  0.07311616728103228,
  0.07601986183481438,
  0.06764652187611615,
  0.07942292306132698,
  0.05797798661881367,
  0.05521092778124083,
  0.051825904575166074,
  0.05748810288443994,
  0.054238120915389126,
  0.028496551633488094,
  0.04145536029952045,
  0.05062448334346494,
  0.03431963206143276,
  0

In [65]:
save_file = {'model' : fine_tuned_model.state_dict(),
             'result': result,
             'config': cfg}

In [68]:
save_keyword = cfg['model_name']
save_path = f'/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/checkpoints/2_{save_keyword}.pt'

In [69]:
torch.save(save_file, save_path)

## Inference

In [ ]:
eval_imgs_dir = f'{eval_folder_dir}/images'
os.listdir(eval_imgs_dir)

In [74]:
eval_labels_path = f'{eval_folder_dir}/info.csv'
eval_csv = pd.read_csv(eval_labels_path)

In [81]:
img_path = []
for file_name in eval_csv['ImageID']:
    file_path = os.path.join(eval_imgs_dir, file_name)
    img_path.append(file_path)
    # print(file_path)

In [88]:
test_csv = pd.DataFrame(img_path)
test_csv.columns = ['img_path']
test_csv

,img_path
0,/opt/ml/repo/level1_imageclassification_cv-lev...
1,/opt/ml/repo/level1_imageclassification_cv-lev...
2,/opt/ml/repo/level1_imageclassification_cv-lev...
3,/opt/ml/repo/level1_imageclassification_cv-lev...
4,/opt/ml/repo/level1_imageclassification_cv-lev...
...,...
12595,/opt/ml/repo/level1_imageclassification_cv-lev...
12596,/opt/ml/repo/level1_imageclassification_cv-lev...
12597,/opt/ml/repo/level1_imageclassification_cv-lev...
12598,/opt/ml/repo/level1_imageclassification_cv-lev...


In [89]:
class TestMaskDataSet(Dataset):#train_labels_path
    def __init__(self, path_df, transforms=None):
        self.transforms = transforms
        self.path_df = path_df
        if transforms is None: # transform이 없으면 자동으로 3개의 transform이 적용됨.
            print('If transforms param is None automatically Resize(224), Normalize, ToTensorV2 Apply')
    
    def __getitem__(self, index):
        img_path = self.path_df['img_path'][index]
        img = np.array(Image.open(img_path))
        # img_class = self.path_df['class'][index]
        if self.transforms is not None:
            img = self.transforms(image=img)["image"]
        else:
            temp_transforms = A.Compose([A.Resize(height=224, width=224), A.Normalize(), ToTensorV2()])
            img = temp_transforms(image=img)["image"]
        return img # , img_class

    def __len__(self):
        return len(self.path_df)

In [90]:
test_transforms = A.Compose([A.Resize(height=224, width=224),
                             A.Normalize(mean=(0.56019358,0.52410121,0.501457),
                                         std=(0.23318603,0.24300033,0.24567522)),
                             ToTensorV2()])

In [97]:
test_dataset = TestMaskDataSet(test_csv, transforms = test_transforms)

In [98]:
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [99]:
def inference(model, test_loader):
    total_test_preds = []
    with torch.no_grad():
        for iter_idx, test_imgs in enumerate(test_loader, start=1):
            model.eval()
            test_imgs = test_imgs.to(device, dtype=torch.float)
            test_logit = model(test_imgs)
            test_preds = test_logit.argmax(dim=-1)
            total_test_preds += test_preds.detach().cpu().numpy().tolist()
    return total_test_preds

In [100]:
test_prediction_list = inference(fine_tuned_model, test_loader)

In [104]:
len(test_prediction_list)

12600

In [107]:
df_submission = pd.read_csv('/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/dataset/eval/submission.csv')
df_submission['ans']=test_prediction_list

In [111]:
(test_prediction_list == df_submission['ans']).sum()

12600

In [113]:
submission_file_path = '/opt/ml/repo/level1_imageclassification_cv-level1-cv-06/T4064/submission'
df_submission.to_csv(submission_file_path + f'/2_{save_keyword}.csv')